# Anti-Drowsiness System 

### Team: Positive Waves

### Importing Required Library

In [1]:
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import cv2
import time
import serial
import pyttsx3

### Creating Function 

In [2]:
def eye_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear
def im_ai():
    engine = pyttsx3.init()
    engine.setProperty("rate", 120)
    engine.say("Hello Driver I am your Anti Drowsiness System Aura ")
    engine.runAndWait()

def check_one(count):
    if count>1:
        ser=serial.Serial('COM7', 9600, timeout=1)
        ser.write(b'H')
        engine = pyttsx3.init()
        engine.setProperty("rate", 158)
        engine.say("Driver in Drowsy")
        engine.runAndWait()
        
def check_zero(count):
        ser=serial.Serial('COM7', 9600, timeout=1)
        ser.write(b'L') 
        if count==15:
            engine = pyttsx3.init()
            engine.setProperty("rate", 158)
            engine.say("Driver is Active")
            engine.runAndWait()

### Capturing and Activating System

In [3]:
threshold = 0.25
check = 70
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["right_eye"]
capaturing=cv2.VideoCapture(0)
flag=0
count_one=0
count_zero=0
im_ai()
while True:
    ret, frame=capaturing.read()
    frame = imutils.resize(frame, width=500)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray, 0)
    for subject in subjects:
        shape = predict(gray, subject)
        shape = face_utils.shape_to_np(shape)
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_ratio(leftEye)
        rightEAR = eye_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        if ear < threshold:
                flag += 1
                if flag >= check:
                    count_one+=1
                    count_zero=0
                    cv2.putText(frame, "***ALERT!***", (10, 30),cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(frame, "Driver in Drowsy", (10,365),cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 0, 255), 2)
                    check_one(count_one)
                    
                elif flag<=check and flag<15:
                    cv2.putText(frame, "***NORMAL***", (10, 30),cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 0), 2)
                    cv2.putText(frame, "Driver is Blinking", (10,365),cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 0), 2)
                else:
                    cv2.putText(frame, "***MODERATE***", (10, 30),cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 0, 0), 2)
                    cv2.putText(frame, "Driver Going to Sleep", (10,365),cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 0, 0), 2)  
        else:
            flag = 0
            count_one=0
            count_zero+=1
            check_zero(count_zero)
            cv2.putText(frame, "***NORMAL***", (10, 30),cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,255,0), 2)
            cv2.putText(frame, "Focus in Driving", (10,365),cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,255, 0), 2)
    cv2.imshow("Drowsiness Window", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        cv2.destroyAllWindows()
        capaturing.release() 
        break

#### ******* END ****** 